In [57]:
import numpy as np
import nibabel as nib
import dsiadapt as dsi
from dipy.viz import fvtk
import dipy.core.gradients as grad
from dipy.data import get_sphere
from dipy.direction import peaks_from_model

%pylab inline
np.set_printoptions(threshold=numpy.nan)

Populating the interactive namespace from numpy and matplotlib


In [58]:
# ODF sphere directions
sphere = get_sphere('symmetric724')
#sphere = sphere.subdivide(2)

In [62]:
fnArr = np.array(['DSI11_invivo_b10k', 'DSI11_invivo_b7k'])
pdfwgtorderArr = np.array([2, 2]);

qgridsz = 65; # Q-space grid size
qgridcenter = qgridsz//2;

for ii in np.arange(fnArr.shape[0]):
    fn = fnArr[ii];
    data = nib.load('data/' + fn + '_roi.nii.gz').get_data();
    gtab = grad.gradient_table('data/' + fn + '_bvals.txt', 'data/' + fn + '_bvecs_dipy.txt');
    mdd = np.loadtxt('data/' + fn + '_stats.txt')[0];
    fov = np.loadtxt('data/' + fn + '_stats.txt')[1];

    rend = mdd / fov * qgridsz;
    dsimodel = dsi.DiffusionSpectrumModel(gtab, qgrid_size=qgridsz, filter_width=np.inf, filter_type='none', r_start=0, r_end=rend, r_step=0.2);
    dsifit = dsimodel.fit(data);
    odf = dsifit.odf(sphere, pdfwgtorderArr[ii]);
    
    dsipeaks = peaks_from_model(dsimodel, data, sphere,
                             relative_peak_threshold=.2,
                             min_separation_angle=15)
    # Save ODF
    r = fvtk.ren();
    fodf_spheres = fvtk.sphere_funcs(odf, sphere, norm=True, scale=2.2);
    #fvtk.add(r, fvtk.sphere_funcs(odf, sphere));
    
    peakvalues = dsipeaks.peak_values / dsipeaks.peak_values.max()
    fodf_peaks = fvtk.peaks(dsipeaks.peak_dirs, peakvalues, scale=2.2)
    fvtk.add(r, fodf_peaks)
    
    fodf_spheres.GetProperty().SetOpacity(0.5)
    fvtk.add(r, fodf_spheres)

    fvtk.camera(r, pos=(0, 1, 0), viewup=(0, 0, 1))
    odfname = 'figure5/' + fn + '_roi_odf_wgt_' + str(pdfwgtorderArr[ii]) + '.png';
    fvtk.record(r, n_frames=1, out_path=odfname, size=(1600,1000))
    #fvtk.show(r)
    


Camera Position (0.00,0.00,1.00)
Camera Focal Point (0.00,0.00,0.00)
Camera View Up (0.00,1.00,0.00)
-------------------------------------
Camera New Position (0.00,1.00,0.00)
Camera New Focal Point (0.00,0.00,0.00)
Camera New View Up (0.00,0.00,1.00)
Camera Position (0.00,0.00,1.00)
Camera Focal Point (0.00,0.00,0.00)
Camera View Up (0.00,1.00,0.00)
-------------------------------------
Camera New Position (0.00,1.00,0.00)
Camera New Focal Point (0.00,0.00,0.00)
Camera New View Up (0.00,0.00,1.00)


In [60]:
dsipeaks.peak_values.max()

91.671292209602001

In [61]:
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)

NameError: name 'csa_model' is not defined